In [15]:
import cv2
import mediapipe as mp
import numpy as np
import time
import ctypes
from pynput.mouse import Controller, Button

# Получение размеров экрана
user32 = ctypes.windll.user32
wScr, hScr = user32.GetSystemMetrics(0), user32.GetSystemMetrics(1)

# Инициализация камеры
cap = cv2.VideoCapture(0)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

# Haar Cascade для распознавания лиц
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Mediapipe — распознавание рук
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.7)
mp_draw = mp.solutions.drawing_utils

mouse = Controller()
smooth_factor = 5
face_detected = False  # Флаг: лицо распознано

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Распознавание лиц
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    if len(faces) > 0:
        face_detected = True
        for (x, y, w, h) in faces:
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
            cv2.putText(frame, 'Лицо распознано', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    else:
        face_detected = False
        cv2.putText(frame, 'Ожидание лица...', (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Если лицо найдено — включаем управление мышью
    if face_detected:
        results = hands.process(rgb_frame)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                lm_list = []
                for id, lm in enumerate(hand_landmarks.landmark):
                    x, y = int(lm.x * width), int(lm.y * height)
                    lm_list.append((x, y))

                if lm_list:
                    index_finger = lm_list[8]
                    pinky_finger = lm_list[20]
                    thumb = lm_list[4]

                    previous_x, previous_y = mouse.position

                    current_x = np.interp(index_finger[0], [0, width], [0, wScr])
                    current_y = np.interp(index_finger[1], [0, height], [0, hScr])

                    screen_x = previous_x + (current_x - previous_x) / smooth_factor
                    screen_y = previous_y + (current_y - previous_y) / smooth_factor

                    mouse.position = (screen_x, screen_y)

                    distance = np.hypot(index_finger[0] - thumb[0], index_finger[1] - thumb[1])
                    if distance < 20:
                        mouse.click(Button.left, 1)
                        time.sleep(0.2)

                    if pinky_finger[1] < index_finger[1] and pinky_finger[1]:
                        mouse.click(Button.right, 1)
                        time.sleep(0.2)

                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    cv2.imshow("Face + Hand Tracking Mouse", frame)
    if cv2.waitKey(1) != -1:
        break

cap.release()
cv2.destroyAllWindows()
